In [4]:
import pandas as pd
import openai
from constants import OPEN_AI_API_KEY

from bert_score import score
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import time

from overlap_evaluate import read_references, read_annotations, ANNOTATIONS, _print_score

from evaluate import *

openai.api_key = OPEN_AI_API_KEY

In [5]:
# # Note production model is davinci
# result = openai.Answer.create(
#     search_model="ada", 
#     model="curie", 
#     question="which puppy is happy?", 
#     documents=["Puppy C is happy."], 
#     examples_context="In 2017, U.S. life expectancy was 78.6 years.", 
#     examples=[["What is human life expectancy in the United States?", "78 years."]], 
#     max_rerank=10,
#     max_tokens=5,
#     stop=["\n", "<|endoftext|>"]
# )


In [6]:
# """
# Things to figure out:

# 1) how many documents to supply GPT-3 (prob 5-10)
# - I say 10, but then set max_rerank to either 5 or 10. 

# 2) What examples to provide?
# -idk. It's almost impossible to evaluate this without running separate experiments. So let's just leave it as-is. 

# 3) What are max_rerank and max_tokens?
# -max_rerank is another reranking layer. I don't actually want it, but I don't think I can remove it either. 
# -max_tokens I think 5 is okay after looking over NQ

# 4) Get a cost estimate based on some examples

# """

In [7]:
sample = """
Abraham Lincoln (/ˈlɪŋkən/; February 12, 1809 – April 15, 1865) was an American lawyer and statesman who served as the 16th president of the United States from 1861 until his assassination in 1865. Lincoln led the nation through the American Civil War and succeeded in preserving the Union, abolishing slavery, bolstering the federal government, and modernizing the U.S. economy. Lincoln was born into poverty in a log cabin in Kentucky and was raised on the frontier, primarily in Indiana. He was self-educated and became a lawyer, Whig Party leader, Illinois state legislator, and U.S. Congressman from Illinois. In 1849, he returned to his law practice but became vexed by the opening of additional lands to slavery as a result of the Kansas–Nebraska Act of 1854. He reentered politics in 1854, becoming a leader in the new Republican Party, and he reached a national audience in the 1858 Senate campaign debates against Stephen Douglas. Lincoln ran for President in 1860, sweeping the North to gain victory. Pro-slavery elements in the South viewed his success as a threat to slavery, and Southern states began seceding from the Union. To secure its independence, the new Confederate States fired on Fort Sumter, a U.S. fort in South Carolina, and Lincoln called up forces to suppress the rebellion and restore the Union. Lincoln, a moderate Republican, had to navigate a contentious array of factions with friends and opponents from both the Democratic and Republican parties. His allies, the War Democrats and the Radical Republicans, demanded harsh treatment of the Southern Confederates. Anti-war Democrats (called "Copperheads") despised Lincoln, and irreconcilable pro-Confederate elements plotted his assassination. He managed the factions by exploiting their mutual enmity, carefully distributing political patronage, and by appealing to the American people. His Gettysburg Address appealed to nationalistic, republican, egalitarian, libertarian, and democratic sentiments. Lincoln supervised the strategy and tactics in the war effort, including the selection of generals, and implemented a naval blockade of the South's trade. He suspended habeas corpus in Maryland, and he averted British intervention by defusing the Trent Affair. He engineered the end to slavery with his Emancipation Proclamation, including his order that the Army and Navy liberate, protect, and recruit former slaves. He also encouraged border states to outlaw slavery, and promoted the Thirteenth Amendment to the United States Constitution, which outlawed slavery across the country. Lincoln managed his own successful re-election campaign. He sought to heal the war-torn nation through reconciliation. On April 14, 1865, just days after the war's end at Appomattox, he was attending a play at Ford's Theatre in Washington, D.C., with his wife Mary when he was fatally shot by Confederate sympathizer John Wilkes Booth. Lincoln is remembered as a martyr and hero of the United States and is often ranked as the greatest president in American history.
"""
sample_examples = [["Who was the 16th president of the United States?", "Abraham Lincoln"], ["Through what major war did Abraham Lincoln serve as United States president?", "American Civil War"], ["What year was Abraham Lincoln killed?", "1865"]]

In [8]:
default_sample = "In 2017, U.S. life expectancy was 78.6 years."

default_sample_examples = [["What is human life expectancy in the United States?", "78 years."]]

In [9]:
tracker = []
class Gpt3QA(QAModel):
    
    def predict_answer(self, question: str, contexts: list) -> str:
        contexts = contexts[0:5]
        contexts = [context['text'] for context in contexts]
        
        result = openai.Answer.create(
            search_model="davinci", 
            model="davinci", 
            question=question, 
            documents=contexts, 
            examples_context=sample, 
            examples=sample_examples, 
            max_rerank=5,
            max_tokens=5,
            stop=["\n", "<|endoftext|>"]
        )
        tracker.append({
            "question": question,
            "result": result
        })

        return result["answers"][0]

In [10]:
model = Gpt3QA(dataset=QADataset.nq, nrows=730)

In [11]:
predictions = model.get_predictions()

Retrieving 730 predictions
At example: 40 after 31.068 seconds. Expecting to take: 8.9321 more minutes
At example: 80 after 60.252 seconds. Expecting to take: 8.1591 more minutes
At example: 120 after 90.153 seconds. Expecting to take: 7.638 more minutes
At example: 160 after 120.362 seconds. Expecting to take: 7.1465 more minutes
At example: 200 after 148.941 seconds. Expecting to take: 6.5782 more minutes
At example: 240 after 178.732 seconds. Expecting to take: 6.0819 more minutes
At example: 280 after 208.096 seconds. Expecting to take: 5.574 more minutes
At example: 320 after 237.606 seconds. Expecting to take: 5.0739 more minutes
At example: 360 after 268.3 seconds. Expecting to take: 4.5959 more minutes
At example: 400 after 296.328 seconds. Expecting to take: 4.0745 more minutes
At example: 440 after 326.5 seconds. Expecting to take: 3.5866 more minutes
At example: 480 after 356.189 seconds. Expecting to take: 3.0919 more minutes
At example: 520 after 386.679 seconds. Expecting

In [12]:
saved_results, label_scores = model.evaluate(get_bert_score=True, get_predictions=False)

Obtaining scores:

Scoring annotation label: total


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scoring annotation label: question_overlap


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scoring annotation label: no_question_overlap


...

Scoring annotation label: no_answer_overlap


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Scoring annotation label: answer_overlap_only


Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


--------------------------------------------------
Label : total
N examples  :  730
Exact Match :  28.08219178082192
Bert Score :  0.47711445205479397
Meteor Score :  0.3123380094167261
--------------------------------------------------
Label : question_overlap
N examples  :  70
Exact Match :  38.57142857142857
Bert Score :  0.5941308571428571
Meteor Score :  0.42076325298536593
--------------------------------------------------
Label : no_question_overlap
N examples  :  141
Exact Match :  21.27659574468085
Bert Score :  0.4132408510638299
Meteor Score :  0.24252388887366658
--------------------------------------------------
Label : answer_overlap
N examples  :  448
Exact Match :  33.92857142857143
Bert Score :  0.547909107142857
Meteor Score :  0.3545561964060013
--------------------------------------------------
Label : no_answer_overlap
N examples  :  282
Exact Match :  18.79432624113475
Bert Score :  0.3646463120567372
Meteor Score :  0.24526798185929582
---------------------------

In [13]:
df = pd.DataFrame(saved_results)

df.to_csv(f"results/nq/gpt3-nq_results.csv", index=False)

In [14]:
import json

with open("results/nq/gpt3-nq_results.json", "w") as outfile:
    json.dump(label_scores, outfile)

In [17]:
saved_results[0]

{'id': 0,
 'answers': ['Wilhelm Conrad Röntgen'],
 'prediction': 'Wilhelm Röntgen',
 'overlap': ['total', 'answer_overlap'],
 'em': False,
 'f1': 0.8,
 'bert_score': 0.90964,
 'meteor_score': 0.3448275862068965}

In [18]:
no_overlap = 0
total_em = 0
for result in saved_results:
    if 'no_answer_overlap' in result['overlap'] and 'no_question_overlap' in result['overlap']:
        no_overlap+=1
        total_em += result['em']

total_em / no_overlap

0.15

In [ ]:
"""

Label : total
N examples  :  50
Exact Match :  24.0
Bert Score :  0.4017627999999999
Meteor Score :  0.3061177000738002
--------------------------------------------------
Label : question_overlap
N examples  :  3
Exact Match :  33.333333333333336
Bert Score :  0.70457
Meteor Score :  0.5842391304347826
--------------------------------------------------
Label : no_question_overlap
N examples  :  7
Exact Match :  14.285714285714286
Bert Score :  0.5535957142857144
Meteor Score :  0.3299637955747401
--------------------------------------------------
Label : answer_overlap
N examples  :  28
Exact Match :  28.571428571428573
Bert Score :  0.4477910714285715
Meteor Score :  0.35637885762034066
--------------------------------------------------
Label : no_answer_overlap
N examples  :  22
Exact Match :  18.181818181818183
Bert Score :  0.3431818181818183
Meteor Score :  0.24214895410547585
--------------------------------------------------
Label : answer_overlap_only
N examples  :  3
Exact Match :  0.0
Bert Score :  0.4876133333333333
Meteor Score :  0.057471264367816084
"""

In [ ]:
"""
Label : total
N examples  :  50
Exact Match :  28.0
Bert Score :  0.456212
Meteor Score :  0.30379518625545215
--------------------------------------------------
Label : question_overlap
N examples  :  3
Exact Match :  66.66666666666667
Bert Score :  0.9144366666666667
Meteor Score :  0.4791666666666667
--------------------------------------------------
Label : no_question_overlap
N examples  :  7
Exact Match :  0.0
Bert Score :  0.48986
Meteor Score :  0.18280980103640201
--------------------------------------------------
Label : answer_overlap
N examples  :  28
Exact Match :  35.714285714285715
Bert Score :  0.5525464285714287
Meteor Score :  0.3562338867936093
--------------------------------------------------
Label : no_answer_overlap
N examples  :  22
Exact Match :  18.181818181818183
Bert Score :  0.33360454545454554
Meteor Score :  0.2370550219341612
--------------------------------------------------
Label : answer_overlap_only
N examples  :  3
Exact Match :  0.0
Bert Score :  0.5882266666666666
Meteor Score :  0.059523809523809514
"""